## Analytical_model

### Import libraries

In [1]:
import pandas as pd
import geopandas as gpd
from sqlalchemy import create_engine
from sqlalchemy import text

#### Create the connection with the DB

In [3]:
# Step 1: Create the connection with the DB
engine = create_engine("postgresql://urbaninfo:@cirrus.ita.chalmers.se/se_tuptp")
conn = engine.connect()

In [4]:
# Step 2: Create a new schema when it is necessary
#schema = '''CREATE SCHEMA name_schema'''
#conn.execute(schema)

#### Multimodal_public_transport_model

In [7]:
# Step 1: Import data
# In this case, the inputs are the outputs of the folder 'pt_4_one_direction_stops_by_name_links_by_mode'.
# These are being used as an example; however, this code serves to create the model of any multimodal public transport network that you can find in the folder '1 - Notebooks_multimodal_models'.
# Links
links_pt_4_query = text('SELECT * FROM pt_4_odirection_ostop_bymode.links_pt_4')
links_pt_4 = gpd.read_postgis(links_pt_4_query, con=engine.connect(), geom_col='geometry')

# Transfers same stop
transfers_same_stops_pt_4_query = text('SELECT * FROM pt_4_odirection_ostop_bymode.transfers_same_stop_pt_4')
transfers_same_stops_pt_4 = gpd.read_postgis(transfers_same_stops_pt_4_query, con=engine.connect(), geom_col='geometry')

# Transfers different stops
transfers_diff_stops_pt_4_query = text('SELECT * FROM pt_4_odirection_ostop_bymode.transfers_diff_stop_pt_4')
transfers_diff_stops_pt_4 = gpd.read_postgis(transfers_diff_stops_pt_4_query, con=engine.connect(), geom_col='geometry')

### Links

In [10]:
# To create the analytical model, it is necessary to choose which frequency, meaning the time of day and which day to select.
# In this example, the frequencies of Monday morning are being used.

# Step 2: Filter the necessary columns of the links table
# The essential columns required to create the analytical model are always the 'source', the 'target', the weight column (in this case, 'time_distance'), and the geometry column.
links_pt_4_monday_morning = links_pt_4[[
    'source', 'target', 'time_distance', 'geometry'
]]

# Step 3: Display the results
links_pt_4_monday_morning.head()

,source,target,time_distance,geometry
0,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,109.0,"LINESTRING (377826.942 6399466.436, 378291.977..."
1,Södra Älvsborgs sjukhus_307955489_bus_service,7:e Villagatan_307955489_bus_service,86.0,"LINESTRING (378640.599 6399758.434, 378291.977..."
2,Borealis kracker_307980814_bus_service,AGA Vattenfall_307980814_bus_service,27.0,"LINESTRING (313106.978 6442246.408, 313191.955..."
3,Jordhammar_307980814_bus_service,AGA Vattenfall_307980814_bus_service,97.0,"LINESTRING (314079.488 6444139.918, 313191.955..."
4,Badet_307959538_bus_service,Abrahamstorp_307959538_bus_service,82.0,"LINESTRING (372200.947 6448247.439, 371017.481..."


### Transfers same stop

In [11]:
# Step 4: Filter the necessary columns of the transfers on the same stop table
# The essential columns required to create the analytical model are always the 'source', the 'target', the weight column (in this case, 'frequency_monday_morning'), and the geometry column.
transfers_same_stops_pt_4_monday_morning = transfers_same_stops_pt_4[[
    'source', 'target', 'freq_monday_morning', 'geometry'
]].rename(columns={
    'freq_monday_morning':'time_distance'
}) # Rename the 'frequency column' with the same name as the weight of the links table (in this case 'time_distance')

# Step 5: Divide the frequency by 2, to get an avarega of the waiting time
transfers_same_stops_pt_4_monday_morning ['time_distance'] = transfers_same_stops_pt_4_monday_morning.time_distance/2

# Step 6: Display the results
transfers_same_stops_pt_4_monday_morning.head()

,source,target,time_distance,geometry
0,7:e Villagatan_307955489_bus_service,7:e Villagatan,NaN,POINT (378291.977 6399668.440)
1,7:e Villagatan,7:e Villagatan_307955489_bus_service,284.210526,POINT (378291.977 6399668.440)
2,AGA Vattenfall_307980814_bus_service,AGA Vattenfall,NaN,POINT (313191.955 6442732.480)
3,AGA Vattenfall,AGA Vattenfall_307980814_bus_service,NaN,POINT (313191.955 6442732.480)
4,Abrahamstorp_307959538_bus_service,Abrahamstorp,NaN,POINT (371017.481 6448504.455)


### Transfers different stops

In [13]:
# Step 7: Filter the necessary columns of the transfers in between different stops
# The essential columns required to create the analytical model are always the 'source', the 'target', the weight column (in this case, 'time_distance', walking time), and the geometry column.
transfers_diff_stops_pt_4_monday_morning = transfers_diff_stops_pt_4[[
    'source', 'target', 'time_distance', 'geometry'
]]

# Step 8: Display the results
transfers_diff_stops_pt_4_monday_morning.head()

,source,target,time_distance,geometry
0,Agerhus,Jordbron,104.669683,"LINESTRING (324621.469 6533086.938, 324604.463..."
1,Alekärrsgatan,Dickson Södra,100.326659,"LINESTRING (321849.955 6403991.951, 321732.984..."
2,Alingsås station,Alingsåsterminalen,110.230024,"LINESTRING (353851.639 6423238.954, 353723.969..."
3,Allégatan,Södra torget,81.020264,"LINESTRING (377468.217 6399358.673, 377421.472..."
4,Allégatan,Åsbogatan,40.696182,"LINESTRING (377468.217 6399358.673, 377515.487..."


### Model

In [14]:
# Step 9: Concatenate the tables in order of generate the analitical model
pt_4_monday_morning_model = pd.concat([
    links_pt_4_monday_morning, transfers_diff_stops_pt_4_monday_morning, transfers_same_stops_pt_4_monday_morning],
    ignore_index=True)

# Step 10: Display the results
pt_4_monday_morning_model.head()

,source,target,time_distance,geometry
0,Bäckängsskolan_307955489_bus_service,7:e Villagatan_307955489_bus_service,109.0,"LINESTRING (377826.942 6399466.436, 378291.977..."
1,Södra Älvsborgs sjukhus_307955489_bus_service,7:e Villagatan_307955489_bus_service,86.0,"LINESTRING (378640.599 6399758.434, 378291.977..."
2,Borealis kracker_307980814_bus_service,AGA Vattenfall_307980814_bus_service,27.0,"LINESTRING (313106.978 6442246.408, 313191.955..."
3,Jordhammar_307980814_bus_service,AGA Vattenfall_307980814_bus_service,97.0,"LINESTRING (314079.488 6444139.918, 313191.955..."
4,Badet_307959538_bus_service,Abrahamstorp_307959538_bus_service,82.0,"LINESTRING (372200.947 6448247.439, 371017.481..."


In [11]:
# Step 11: Set geometry and export to the DB
pt_4_monday_morning_model = gpd.GeoDataFrame(pt_4_monday_morning_model, geometry='geometry', crs='3006')
pt_4_monday_morning_model.to_postgis('pt_4_monday_morning_model', engine, schema='pt_4_odirection_ostop_bymode', if_exists ='replace')